In [3]:
import itertools
from queue import LifoQueue
import csv
import os
import pprint
import re
import time
from collections import defaultdict
from itertools import combinations, product

import pandas as pd
from gensim.models import Word2Vec, KeyedVectors


In [4]:
%load_ext line_profiler

ModuleNotFoundError: No module named 'line_profiler'

In [5]:
def read_data():
    DATA_PATH = r'medicalTextTrees/parus_results'
    files = os.listdir(DATA_PATH)
    df_columns = ['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel']
    # trees_df = pd.DataFrame(columns=df_columns)
    full_df = []
    stable_df = []
    many_roots_df = []
    very_long_df = []
    for file in files:
        full_dir = os.path.join(DATA_PATH, file)
        name = file.split('.')[0]
        with open(full_dir, encoding='utf-8') as f:
            this_df = pd.read_csv(f, sep='\t', names=df_columns)
            if this_df['id'].duplicated().any():
                start_of_subtree_df = list(this_df.groupby(this_df.id).get_group(1).index)
                boundaries = start_of_subtree_df + [max(list(this_df.index)) + 1]
                list_of_dfs = [this_df.iloc[boundaries[n]:boundaries[n + 1]] for n in range(len(boundaries) - 1)]
                local_counter = 1
                for df in list_of_dfs:
                    df['sent_name'] = name + '_' + str(local_counter)
                    full_df.append(df)
                    stable_df.append(df)
                    # trees_df = pd.concat([trees_df, df], ignore_index=True)
                    local_counter += 1
            else:
                this_df['sent_name'] = name
                if this_df.groupby(this_df.deprel).get_group('ROOT').shape[0] > 1:
                    many_roots_df.append(this_df)
                elif this_df.shape[0] > 21:
                    very_long_df.append(this_df)
                else:
                    stable_df.append(this_df)
                # trees_df = pd.concat([trees_df, this_df], ignore_index=True)
                full_df.append(this_df)
    trees_df = pd.concat(stable_df, axis=0, ignore_index=True)
    # delete useless data
    # trees_df = trees_df.drop(columns=['upostag', 'xpostag', 'feats'], axis=1)
    trees_df = trees_df.drop(columns=['xpostag', 'feats'], axis=1)
    # trees_df.drop(index=[11067], inplace=True)
    trees_df.loc[13742, 'deprel'] = 'разъяснит'

    # delete relations of type PUNC and reindex
    trees_df_filtered = trees_df[trees_df.deprel != 'PUNC']
    # trees_df_filtered = trees_df_filtered.loc[trees_df_filtered['sent_name'] != '37918_12']
    # trees_df_filtered = trees_df_filtered.loc[trees_df_filtered['sent_name'] != '38897_9'] # 1) very long sentence 2) 5 roots
    trees_df_filtered = trees_df_filtered.reset_index(drop=True)
    trees_df_filtered.index = trees_df_filtered.index + 1
    # trees_df_filtered.loc[12239, 'deprel'] = '1-компл'
    # trees_df_filtered.loc[12239, 'head'] = 2

    trees_long_df = pd.concat(very_long_df, axis=0, ignore_index=True)
    trees_roots_df = pd.concat(many_roots_df, axis=0, ignore_index=True)
    trees_long_df = trees_long_df[trees_long_df.deprel != 'PUNC']
    trees_roots_df = trees_roots_df[trees_roots_df.deprel != 'PUNC']
    trees_long_df = trees_long_df.reset_index(drop=True)
    trees_long_df.index = trees_long_df.index + 1
    trees_roots_df = trees_roots_df.reset_index(drop=True)
    trees_roots_df.index = trees_roots_df.index + 1

    trees_full_df = pd.concat(full_df, axis=0, ignore_index=True)
    trees_full_df = trees_full_df.reset_index(drop=True)
    trees_full_df.index = trees_full_df.index + 1
    trees_full_df.drop(columns=['upostag', 'xpostag', 'feats'], axis=1)
    trees_full_df = trees_full_df[trees_full_df.deprel != 'PUNC']

    replaced_numbers = [k for k, v in trees_full_df.lemma.str.contains('#').to_dict().items() if
                        v == True]  # едленно, вставить выше
    for num in replaced_numbers:
        trees_df_filtered.loc[num, 'upostag'] = 'Num'
        trees_full_df.loc[num, 'upostag'] = 'Num'

    target_sents = list({'55338_41', '58401_7', '32384_8', '31736_14', '48714_8', '54996_6'}) # TEST
    target_sents = list({'55338_41', '58401_7'})  # TEST
    trees_df_filtered = trees_df_filtered.loc[trees_df_filtered.sent_name.isin(target_sents)] # TEST

    # trees_full_df.loc[trees_full_df.index.isin(replaced_numbers)].assign(upostag = 'N')
    return trees_full_df, trees_df_filtered

In [6]:
def train_word2vec(trees_df_filtered, lemmas, dict_lemmas_3, part_of_speech_node_id):
    lemma_sent_df = trees_df_filtered[['lemma', 'sent_name']]
    lemma_sent_dict = {}
    for name, group in lemma_sent_df.groupby('sent_name'):
        lemma_sent_dict[name] = []
        for _, row in group.iterrows():
            lemma_sent_dict[name].append(row['lemma'])
    sentences = list(lemma_sent_dict.values())

    only_medical = Word2Vec(sentences, min_count=1)
    # X = only_medical[only_medical.wv.vocab]
    # pca = PCA(n_components=2)
    # result = pca.fit_transform(X)
    # # create a scatter plot of the projection
    # pyplot.scatter(result[:, 0], result[:, 1])
    # words = list(only_medical.wv.vocab)
    # for i, word in enumerate(words):
    #     pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))

    model_2 = Word2Vec(min_count=1)
    model_2.build_vocab(sentences)
    w2v_fpath = "additionalCorpus/all_norm-sz100-w10-cb0-it1-min100.w2v"

    # model.intersect_word2vec_format
    # similar_dict = {}
    # for lemma in lemmas:
    #     similar_dict[lemma] = model.most_similar(lemma)
    # embeddings = [model[i] for i in lemmas]

    # additional_model =
    # w2v = Word2Vec.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
    upper_bound = 50000
    high_cosine_dist = 0.8
    model = KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
    model_2.build_vocab([list(model.vocab.keys())[:upper_bound]], update=True)
    model_2.intersect_word2vec_format(w2v_fpath, binary=True, lockf=1.0, unicode_errors='ignore')
    model_2.train(sentences, total_examples=upper_bound, epochs=model_2.iter)
    similar_dict = {lemma: model_2.most_similar(lemma, topn=15) for lemma in lemmas if not pattern.match(lemma)}
    similar_lemmas_dict = {}
    for lemma, similar_lemmas in similar_dict.items():
        for similar_lemma, cosine_dist in similar_lemmas:
            if cosine_dist > high_cosine_dist and similar_lemma in lemmas.keys() and part_of_speech_node_id[similar_lemma] == part_of_speech_node_id[lemma]:
                if lemma not in similar_lemmas_dict.keys():
                    similar_lemmas_dict[lemma] = [similar_lemma]
                else:
                    similar_lemmas_dict[lemma].append(similar_lemma)
    all_values = [item for sublist in similar_lemmas_dict.values() for item in sublist]
    most_freq = set([i for i in all_values if all_values.count(i) > 11])
    similar_lemmas_dict_filtered = {}
    for k, v in similar_lemmas_dict.items():
        stable = set(v) - most_freq
        if 0 < len(stable) <= 10:
            similar_lemmas_dict_filtered[k] = stable

    pprint.pprint(similar_lemmas_dict_filtered)
    # similar_mapping = {}
    for lemma, similar_lemmas in similar_lemmas_dict_filtered.items():
        for similar_lemma in similar_lemmas:
            lemmas[lemma].append(lemmas[similar_lemma][0])
            # if lemmas[similar_lemma][0] not in similar_mapping.keys():
            #     similar_mapping[lemmas[similar_lemma][0]] = [lemma]
            # else:
            #     similar_mapping[lemmas[similar_lemma][0]].append(lemma)
    pprint.pprint(lemmas)

    for lemma, similar_lemmas in similar_lemmas_dict_filtered.items():
        for similar_lemma in similar_lemmas:
            if lemma in dict_lemmas_3.keys():
                dict_lemmas_3[lemma].append(lemmas[similar_lemma][0])


In [7]:
class Node:
    def __init__(self, id, lemma=None, form=None, sent_name=None, is_included=False):
        self.id = id
        self.lemma = lemma
        self.form = form
        self.sent_name = sent_name
        self.is_included = is_included


class Edge:
    def __init__(self, node_id_from, node_id_to, weight):
        self.node_from = node_id_from
        self.node_to = node_id_to
        self.weight = weight  # relation type

In [8]:
class Tree:

    def __init__(self):
        self.edges = []
        self.nodes = []
        self.inactive = set()
        self.created = set()
        self.heights = {}
        self.edges_dict_from = {}
        self.edges_dict_to = {}
        self.nodes_dict_id = {}
        self.additional_nodes = set()
        self.similar_lemmas = {}
        self.global_similar_mapping = {}

    def set_help_dict(self):
        self.edges_dict_from = {k: list(v) for k, v in itertools.groupby(sorted(self.edges, key=lambda x: x.node_from),
                                                                         key=lambda x: x.node_from)}
        self.nodes_dict_id = {node.id: node for node in self.nodes}
        self.edges_dict_to = {k: list(v) for k, v in itertools.groupby(self.edges, key=lambda x: x.node_to)}
        # filtered_edges = list(filter(lambda x: x.node_to not in self.additional_nodes and x.node_from not in self.additional_nodes, self.edges))
        # self.edges_dict_to = {k: list(set(v) - self.additional_nodes) for k, v in itertools.groupby(filtered_edges, key=lambda x: x.node_to)}

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def get_node(self, node_id):
        return self.nodes_dict_id.get(node_id)  # return Node class instance

    def get_children(self, node_id):
        edges = self.edges_dict_from.get(node_id)
        # only_active = list(filter(lambda edge: edge.node_to not in self.inactive and edge.node_from not in self.inactive, edges if edges is not None else []))
        # return list(map(lambda x: x.node_to, only_active if only_active is not None else []))
        return set(map(lambda x: x.node_to, edges if edges is not None else []))

    def get_edge(self, to_id):
        return self.edges_dict_to.get(to_id)

    def remove_edge(self, to_id):
        self.edges = list(filter(lambda x: x.node_to != to_id, self.edges))

    def copy_node_details(self, existing_node):
        new_node = Node(id=len(self.nodes),
                    form=existing_node.form,
                    sent_name=existing_node.sent_name,
                    is_included=existing_node.is_included)
        self.nodes_dict_id[new_node.id] = new_node
        self.created.add(new_node.id)
        return new_node

    def add_new_edges(self, new_node_id, children):
        for child_id in children:
            new_edge = Edge(new_node_id, child_id, self.get_edge(child_id)[0].weight)
            if child_id in self.edges_dict_to.keys():
                self.edges_dict_to[child_id].append(new_edge)
            else:
                self.edges_dict_to[child_id] = [new_edge]
            self.edges.append(new_edge)
            if new_node_id in self.edges_dict_from.keys():
                self.edges_dict_from[new_node_id].append(new_edge)
            else:
                self.edges_dict_from[new_node_id] = [new_edge]

    def add_edge_to_dict(self, edge):
        self.edges_dict_to[edge.node_to] = [edge]
        if edge.node_from in self.edges_dict_from.keys():
            self.edges_dict_from[edge.node_from].append(edge)
        else:
            self.edges_dict_from[edge.node_from] = edge
        self.edges.append(edge)

    def add_node_to_dict(self, node):
        self.nodes_dict_id[node.id] = node
        self.nodes.append(node)

    def add_inactive(self, node_id):
        self.inactive.add(node_id)

    def get_children_nodes(self, children):
        children_nodes = {
            str(Tree.get_edge(self, child)[0].weight) + str(Tree.get_node(self, child).lemma): child for
            child in children}
        return children_nodes
    
    def calculate_heights(self):
        # visited = np.full(len(self.nodes), False, dtype=bool)
        visited = {node.id: False for node in self.nodes}
        # self.heights = np.full(len(self.nodes), -1, dtype=int)  # all heights are -1 initially
        stack = LifoQueue()
        stack.put(0)
        prev = None
        while stack.qsize() > 0:
            curr = stack.get()
            stack.put(curr)
            if not visited[curr]:
                visited[curr] = True
            children = self.get_children(curr)
            if len(children) == 0:
                self.heights[curr] = [0]
                prev = curr
                stack.get()
            else:
                all_visited_flag = True
                children_filtered = set(children) - self.additional_nodes
                for child in children_filtered:
                    try:
                        if not visited[child]:
                            all_visited_flag = False
                            stack.put(child)
                    except IndexError as ie:
                        ff = []
                if all_visited_flag:
                    curr_height = []
                    if len(children_filtered) > 1:
                        for child in children_filtered:
                            for child_height in self.heights[child]:
                                curr_height.append(child_height + 1)
                    else:
                        try:
                            curr_height = [h + 1 for h in self.heights[prev]]
                        except KeyError as e:
                            efl = []
                    self.heights[curr] = list(set(curr_height))
                    prev = curr
                    stack.get()
    def simple_dfs(self, vertex, subtree_vertices):
        sequence = []
        node = self.get_node(vertex)
        if node is not None:
            sequence.append(tuple([vertex, node.lemma, node.form, node.sent_name]))
            # visited = np.full(len(self.nodes), False, dtype=bool)
            visited = {node.id: False for node in self.nodes}
            stack = [vertex]
            while len(stack) > 0:
                curr = stack[-1]
                if not visited[curr]:
                    visited[curr] = True
                children = self.get_children(curr)
                if len(children) == 0:
                    stack.pop()
                else:
                    all_visited_flag = True
                    for child in children:
                        if child in subtree_vertices and not visited[child]:
                            all_visited_flag = False
                            stack.append(child)
                            node = self.get_node(child)
                            sequence.append(tuple([child, node.lemma, node.form, node.sent_name]))
                    if all_visited_flag:
                        stack.pop()
        return sequence

In [9]:
def construct_tree(trees_df_filtered, dict_lemmas, dict_rel, dict_lemmas_rev):
    # construct a tree with a list of edges and a list of nodes
    whole_tree = Tree()
    root_node = Node(0, 0)  # add root
    Tree.add_node(whole_tree, root_node)
    # new_id_count = len(trees_df_filtered) + 1
    new_id_count = 1
    similar_lemmas_dict = {}
    global_similar_mapping = {}
    for name, group in trees_df_filtered.groupby('sent_name'):
        row_ids = trees_df_filtered.index[trees_df_filtered.sent_name == name].to_list()
        # temporary dictionary for remapping indices
        temp_dict = {key: row_ids[ind] for ind, key in enumerate(group.id.to_list())}
        temp_dict[0] = 0
        children_dict = {}
        edge_to_weight = {}
        for _, row in group.iterrows():
            # parameters for main node
            curr_lemmas = dict_lemmas.get(row['lemma'])
            main_id = temp_dict.get(row['id'])
            from_id = temp_dict.get(row['head'])
            weight = dict_rel.get(row['deprel'])
            sent = row['sent_name']
            form = row['form']
            # create main node
            create_new_node(whole_tree, main_id, curr_lemmas[0], form, sent, weight, from_id)
            children = [main_id]
            edge_to_weight[main_id] = weight
            global_similar_mapping[main_id] = main_id
            # if lemma has additional values add additional nodes
            if len(curr_lemmas) > 1:
                for i in range(1, len(curr_lemmas)):
                    new_id_count += 1
                    while new_id_count in list(map(lambda x: x.id, whole_tree.nodes)):
                        new_id_count += 1
                    create_new_node(whole_tree, new_id_count, curr_lemmas[i], form, sent, weight, from_id)
                    edge_to_weight[new_id_count] = weight
                    if main_id not in similar_lemmas_dict.keys():
                        similar_lemmas_dict[main_id] = [new_id_count]
                    else:
                        similar_lemmas_dict[main_id].append(new_id_count)
                    global_similar_mapping[new_id_count] = main_id
                    children.append(new_id_count)
            if from_id not in children_dict.keys():
                children_dict[from_id] = children
            else:
                children_dict[from_id].extend(children)
        # if parent has additional values add additional edges
        for from_id, children in children_dict.items():
            if from_id in similar_lemmas_dict.keys():
                similar_ids = similar_lemmas_dict[from_id]
                for similar_id in similar_ids:
                    for child_id in children:
                        Tree.add_edge(whole_tree, Edge(similar_id, child_id, edge_to_weight[child_id]))
        #
        # for main_id, similar_ids in similar_lemmas_dict.items():
        #     if main_id in children_dict.keys():
        #         for child in children_dict[main_id]:
        #             for similar_id in similar_ids:
        #                 Tree.add_edge(whole_tree, Edge(similar_id, child, edge_to_weight[main_id]))
    whole_tree.additional_nodes = set([sublist for list in similar_lemmas_dict.values() for sublist in list])
    whole_tree.similar_lemmas = similar_lemmas_dict
    whole_tree.global_similar_mapping = global_similar_mapping
    return whole_tree

In [10]:
def add_children_to_parents(k_2, filtered_groups, whole_tree, curr_height, old_node_new_nodes):
    all_parents = set()
    for k, v in filtered_groups.items():
        for v_id in list(v):
            edge_to_curr = Tree.get_edge(whole_tree, v_id)
            Tree.get_node(whole_tree, v_id).is_included = True
            if edge_to_curr is not None:
                parent = edge_to_curr[0].node_from
                try:
                    if parent not in whole_tree.heights.keys(): # same as in whole_tree.additional
                        parent = whole_tree.global_similar_mapping[parent]
                    if max(whole_tree.heights[parent]) > curr_height:
                        all_parents.add(parent)
                except KeyError as ke:
                    dfkdf = []
                if v_id in old_node_new_nodes.keys():
                    lemmas_to_visit = old_node_new_nodes[v_id]
                else:
                    lemmas_to_visit = [k]
                if v_id in whole_tree.similar_lemmas.keys():
                    for node_id in whole_tree.similar_lemmas[v_id]:
                        lemmas_to_visit.append(Tree.get_node(whole_tree, node_id).lemma)
                for lemma in lemmas_to_visit:
                    label_for_child = str(edge_to_curr[0].weight) + str(lemma)
                    if parent not in k_2.keys():
                        k_2[parent] = {label_for_child}
                    else:
                        k_2[parent].add(label_for_child)
    return all_parents

In [11]:
def add_additional_children_to_parents(k_2, whole_tree, all_parents):
    additional_child_nodes = {}
    for parent in all_parents:
        for child_id in Tree.get_children(whole_tree, parent):# - whole_tree.additional_nodes:
            edge_to_curr = Tree.get_edge(whole_tree, child_id)[0]
            child_node = Tree.get_node(whole_tree, child_id)
            if not child_node.is_included:
                label_for_child = str(edge_to_curr.weight) + str(child_node.lemma)
                k_2[parent].add(label_for_child)
                child_node.is_included = True
                if parent not in additional_child_nodes.keys():
                    additional_child_nodes[label_for_child] = child_id
                else:
                    additional_child_nodes[label_for_child].append(child_id)
    return additional_child_nodes

In [12]:
def produce_combinations(k_2, v_id, str_sequence_help, str_sequence_help_reversed, equal_nodes, equal_nodes_mapping):
    if len(equal_nodes) > 0:
        list_for_combinations = []
        prepared_k_2 = set()
        included_labels = []
        for child_tree in k_2[v_id]:
            if child_tree in [item for sublist in list(equal_nodes.values()) for item in
                              sublist] or child_tree in equal_nodes.keys():
                if child_tree in equal_nodes_mapping.keys():
                    actual_label = equal_nodes_mapping[child_tree]
                else:
                    actual_label = child_tree
                if actual_label not in included_labels:
                    if actual_label in equal_nodes.keys():
                        list_for_combinations.append(equal_nodes[actual_label])
                    else:
                        list_for_combinations.append(equal_nodes[child_tree])
                    included_labels.append(actual_label)
            else:
                prepared_k_2.add(child_tree)
        combinations_repeated = list(product(*(list_for_combinations)))
        all_combinations = []
        if len(prepared_k_2) > 0:
            for l in combinations_repeated:
                merged = list(l) + list(prepared_k_2)
                all_combinations.extend(list(combinations(merged, i)) for i in range(1, len(merged) + 1))
        else:
            for l in combinations_repeated:
                all_combinations.extend(list(combinations(list(l), i)) for i in range(1, len(list(l)) + 1))
    else:
        list_for_combinations = k_2[v_id]
        all_combinations = [list(combinations(list_for_combinations, i)) for i in
                            range(1, len(list_for_combinations) + 1)]
    all_combinations_str_joined = set()
    for comb in all_combinations:
        for tup in comb:
            combs = [str(item) for item in sorted(list(tup))]
            joined_label = EMPTY_STR.join(combs)
            if joined_label not in str_sequence_help.keys():
                str_sequence_help[joined_label] = combs
                str_sequence_help_reversed[tuple(combs)] = joined_label
            all_combinations_str_joined.add(joined_label)
    return all_combinations_str_joined

In [13]:
def get_nodeid_repeats(filtered_combination_ids, str_sequence_help):
    dict_nodeid_comb = {}
    for k, v in filtered_combination_ids.items():
        for v_i in v:
            if v_i in dict_nodeid_comb.keys():
                dict_nodeid_comb[v_i].append(str_sequence_help.get(k))
            else:
                dict_nodeid_comb[v_i] = [str_sequence_help.get(k)]
    return dict_nodeid_comb


In [14]:
def compute_part_new_new(whole_tree, lemma_count, grouped_heights):
    classes_subtreeid_nodes = {}
    classes_subtreeid_nodes_list = {}
    unique_subtrees_mapped_global_subtree_lemma = {}
    old_node_new_nodes = {}
    equal_nodes_mapping = {}
    subtree_label_sent = {}
    k_2 = {}  # identifiers of edges of subtrees
    lemma_nodeid_dict = {}
    saved_combinations = []
    for nodes in grouped_heights:
        curr_height = nodes[0]
        print(curr_height)
        start = time.time()
        id_lemma_dict = {node.id: node.lemma for node in nodes[1]}
        grouped_lemmas = defaultdict(list)
        for key, value in id_lemma_dict.items():
            grouped_lemmas[value].append(key)
        all_parents = add_children_to_parents(k_2, grouped_lemmas, whole_tree, curr_height, old_node_new_nodes)
        additional_child_nodes = add_additional_children_to_parents(k_2, whole_tree, all_parents)
        for additional_child, child_id in additional_child_nodes.items():
            if additional_child not in lemma_nodeid_dict.keys():
                lemma_nodeid_dict[additional_child] = {child_id}
            else:
                lemma_nodeid_dict[additional_child].add(child_id)
        for lemma, ids in grouped_lemmas.items():
            for v_id in ids:
                edge_to_curr = Tree.get_edge(whole_tree, v_id)
                if edge_to_curr is not None:
                    label_for_child = str(edge_to_curr[0].weight) + str(lemma)
                    if label_for_child not in lemma_nodeid_dict.keys():
                        lemma_nodeid_dict[label_for_child] = {v_id}
                    else:
                        lemma_nodeid_dict[label_for_child].add(v_id)

        if curr_height != 0:  # not applicable to leaves, leaves don't have subtrees
            filtered_groups = {k: v for k, v in grouped_lemmas.items() if len(v) > 1}
            for lemma, ids in filtered_groups.items():
                combination_ids = {}
                str_sequence_help = {}
                str_sequence_help_reversed = {}

                # generate combinations
                for v_id in ids:
                    equal_nodes = {}
                    # only for duplicating nodes
                    children = Tree.get_children(whole_tree, v_id) - whole_tree.created - whole_tree.additional_nodes
                    for child in children:
                        if child in old_node_new_nodes.keys() or child in whole_tree.similar_lemmas.keys():
                            edge_to_child = Tree.get_edge(whole_tree, child)[0]
                            child_node = Tree.get_node(whole_tree, child)
                            w = str(edge_to_child.weight)
                            actual_label = w + str(child_node.lemma)
                            merge = []
                            if child in old_node_new_nodes.keys():
                                for l in old_node_new_nodes[child]: # refactor this and below
                                    new_label = w + str(l)
                                    merge.append(new_label)
                                    equal_nodes_mapping[new_label] = actual_label
                            else:
                                merge.append(actual_label)
                                for node_id in whole_tree.similar_lemmas[child]:
                                    new_label = w + str(Tree.get_node(whole_tree, node_id).lemma)
                                    merge.append(new_label)
                                    equal_nodes_mapping[new_label] = actual_label
                            if actual_label not in equal_nodes.keys():
                                equal_nodes[actual_label] = merge
                            else:
                                equal_nodes[actual_label].extend(merge)
                    all_combinations_str_joined = produce_combinations(k_2, v_id, str_sequence_help,
                                                                       str_sequence_help_reversed, equal_nodes,
                                                                       equal_nodes_mapping)
                    for label in all_combinations_str_joined:
                        if label in combination_ids.keys():
                            combination_ids[label].append(v_id)
                        else:
                            combination_ids[label] = [v_id]

                filtered_combination_ids = {k: v for k, v in combination_ids.items() if len(v) > 1}
                for tree_label, node_list in filtered_combination_ids.items():
                    str_tree_label = str(lemma) + tree_label
                    if str_tree_label not in unique_subtrees_mapped_global_subtree_lemma.keys():
                        unique_subtrees_mapped_global_subtree_lemma[str_tree_label] = lemma_count
                        lemma_count += 1
                # 16: [['107', '919'], ['208', '919'], ['919'], ['107'], ['208']]
                dict_nodeid_comb = get_nodeid_repeats(filtered_combination_ids, str_sequence_help)
                for node_id, node_subtrees in dict_nodeid_comb.items():
                    existing_node = Tree.get_node(whole_tree, node_id)
                    edge_to_curr = Tree.get_edge(whole_tree, node_id)[0]
                    children = Tree.get_children(whole_tree, node_id)
                    for subtree in node_subtrees:
                        subtree_text = str_sequence_help_reversed.get(tuple(subtree))
                        subtree_new_label = unique_subtrees_mapped_global_subtree_lemma.get(str(lemma) + subtree_text)

                        if subtree_new_label not in subtree_label_sent.keys() or existing_node.sent_name not in subtree_label_sent[subtree_new_label]:
                            # add new node with a new lemma
                            new_node = Tree.copy_node_details(whole_tree, existing_node)
                            new_node.lemma = subtree_new_label
                            Tree.add_node_to_dict(whole_tree, new_node)

                            if subtree_new_label not in subtree_label_sent.keys():
                                subtree_label_sent[subtree_new_label] = [existing_node.sent_name]
                            else:
                                subtree_label_sent[subtree_new_label].append(existing_node.sent_name)

                            # add new node to node aliases
                            if node_id not in old_node_new_nodes.keys():
                                old_node_new_nodes[node_id] = [new_node.lemma]
                            else:
                                old_node_new_nodes[node_id].append(new_node.lemma)

                            # add an edge to it
                            edge = Edge(edge_to_curr.node_from, new_node.id, edge_to_curr.weight)
                            Tree.add_edge_to_dict(whole_tree, edge)

                            # add new node to parent
                            parent_subtree_text = str(edge_to_curr.weight) + str(subtree_new_label)
                            if parent_subtree_text not in lemma_nodeid_dict.keys():
                                lemma_nodeid_dict[parent_subtree_text] = {new_node.id}
                            else:
                                lemma_nodeid_dict[parent_subtree_text].add(new_node.id)

                            subtree_children = []
                            children_nodes = {}
                            for subtree_node in subtree:
                                if subtree_node not in lemma_nodeid_dict.keys():
                                    if subtree_node in equal_nodes_mapping.keys():
                                        target_child = \
                                        list(set(lemma_nodeid_dict[equal_nodes_mapping[subtree_node]]) & children)[0]
                                    else:
                                        if len(children_nodes) == 0:
                                            children_nodes = Tree.get_children_nodes(whole_tree, children)
                                        target_child = children_nodes[subtree_node]
                                else:
                                    intersection = set(lemma_nodeid_dict[subtree_node]) & children
                                    if len(intersection) == 0:
                                        if len(children_nodes) == 0:
                                            children_nodes = Tree.get_children_nodes(whole_tree, children)
                                        try:
                                            target_child = children_nodes[subtree_node]
                                        except KeyError as ke:
                                            gh = []
                                    else:
                                        target_child = list(intersection)[0]
                                subtree_children.append(target_child)

                            if len(subtree_children) > 0:
                                general_comb = ''.join(sorted([str(whole_tree.global_similar_mapping[child_id]) for child_id in subtree_children]))
                                if general_comb not in saved_combinations:
                                    # add edges to subtree's children from new node
                                    Tree.add_new_edges(whole_tree, new_node.id, subtree_children)

                                    # assign class
                                    if subtree_new_label not in classes_subtreeid_nodes.keys():
                                        classes_subtreeid_nodes[subtree_new_label] = [new_node.id]
                                    else:
                                        classes_subtreeid_nodes[subtree_new_label].append(new_node.id)

                                    subtree_deep_children = set()
                                    for subtree_lemma in list(
                                            map(lambda x: Tree.get_node(whole_tree, x).lemma, subtree_children)):
                                        if subtree_lemma in classes_subtreeid_nodes_list.keys():
                                            subtree_deep_children.update(classes_subtreeid_nodes_list[subtree_lemma])
                                    subtree_deep_children.update(subtree_children)
                                    only_active = subtree_deep_children - whole_tree.inactive
                                    # only_active = (whole_tree.inactive.symmetric_difference(set_children))&set_children

                                    if subtree_new_label not in classes_subtreeid_nodes_list.keys():
                                        classes_subtreeid_nodes_list[subtree_new_label] = only_active
                                    else:
                                        classes_subtreeid_nodes_list[subtree_new_label].update(only_active)
                                    classes_subtreeid_nodes_list[subtree_new_label].add(new_node.id)

                                    saved_combinations.append(general_comb)

                    # remove old node and edges to/from it
                    Tree.add_inactive(whole_tree, node_id)
        print(time.time() - start)
    classes_subtreeid_nodes = {k: v for k, v in classes_subtreeid_nodes.items() if len(v) > 1} # TODO: why do len=1 entries even appear here??
    return classes_subtreeid_nodes, classes_subtreeid_nodes_list


In [166]:
%lprun -f read_data read_data()

<ipython-input-165-40daba35bc3a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sent_name'] = name + '_' + str(local_counter)


In [162]:
read_data()

<ipython-input-156-0571692d4150>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sent_name'] = name + '_' + str(local_counter)


,id,form,lemma,head,deprel,sent_name
1,1,Около,около,4,предл,52123_9
2,2,10,#число,3,количест,52123_9
3,3,лет,год,1,предл,52123_9
4,4,назад,назад,5,обст,52123_9
5,5,ЧМТ,ЧМТ,0,ROOT,52123_9
...,...,...,...,...,...,...
15255,14,в,в,17,обст,34695_0
15256,15,армии,армия,14,предл,34695_0
15257,16,не,не,17,огранич,34695_0
15258,17,служил,служить,1,сочин,34695_0


In [16]:
start = time.time()
trees_full_df, trees_df_filtered = read_data()
test_3_sent = trees_df_filtered.head(12)
print('Time on reading the data: ' + str(time.time() - start))

part_of_speech_node_id = dict(trees_full_df[['lemma', 'upostag']].groupby(['lemma', 'upostag']).groups.keys())

# get all lemmas and create a dictionary to map to numbers
dict_lemmas = {lemma: [index] for index, lemma in enumerate(dict.fromkeys(trees_df_filtered['lemma'].to_list()), 1)}
dict_lemmas_full = {lemma: [index] for index, lemma in
                        enumerate(dict.fromkeys(trees_full_df['lemma'].to_list()), 1)}
dict_lemmas_rev = {index[0]: lemma for lemma, index in dict_lemmas_full.items()}
dict_lemmas_3 = {lemma: [index] for index, lemma in enumerate(dict.fromkeys(test_3_sent['lemma'].to_list()), 1)}
    #
numbers = [item for item in list(dict_lemmas_full.keys()) if pattern.match(item)]
numbers_one_lemma = dict_lemmas_full[numbers[0]]
for num in numbers:
    dict_lemmas[num] = numbers_one_lemma
    dict_lemmas_full[num] = numbers_one_lemma
    if num in dict_lemmas_3.keys():
        dict_lemmas_3[num] = numbers_one_lemma
# get all relations and create a dictionary to map to numbers
# dict_rel = {rel: index for index, rel in enumerate(dict.fromkeys(trees_full_df['deprel'].to_list()))}
dict_rel = {rel: index for index, rel in enumerate(dict.fromkeys(trees_df_filtered['deprel'].to_list()))}
train_word2vec(trees_full_df, dict_lemmas_full, dict_lemmas, part_of_speech_node_id)

    #
    # test_dict_lemmas = get_test_dict_lemmas()
    # new_test_dict = {k: test_dict_lemmas[i] for i, k in enumerate(dict_lemmas_3.keys())}
#     whole_tree = construct_tree(trees_df_filtered, dict_lemmas, dict_rel, dict_lemmas_rev)
    # write_tree_in_table(whole_tree)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'medicalTextTrees/parus_results'

In [ ]:
%lprun -f construct_tree construct_tree(trees_df_filtered, dict_lemmas, dict_rel, dict_lemmas_rev)

In [ ]:
 print('Time on constructing the tree: ' + str(time.time() - start))
    # whole_tree = new_test()
    Tree.set_help_dict(whole_tree)
    # partition nodes by height
    start = time.time()
    Tree.calculate_heights(whole_tree)
    print('Time on calculating all heights: ' + str(time.time() - start))

    heights_dictionary = {Tree.get_node(whole_tree, node_id): heights for node_id, heights in
                          whole_tree.heights.items()}
    grouped_heights = defaultdict(list)
    for node_1, heights in heights_dictionary.items():
        for height in heights:
            grouped_heights[height].append(node_1)
    grouped_heights = sorted(grouped_heights.items(), key=lambda x: x[0])

    dict_lemmas_size = max(set(map(lambda x: x.lemma, whole_tree.nodes)))

    # classes for partial repeats
    start = time.time()
    classes_part, classes_part_list = compute_part_new_new(whole_tree, dict_lemmas_size, grouped_heights)
    write_tree_in_table(whole_tree)
    # classes_part = compute_part_subtrees(whole_tree, dict_lemmas_size, grouped_heights)
    print('Time on calculating partial repeats: ' + str(time.time() - start))

In [161]:
start = time.time()
#classes_part = compute_part_new_new(whole_tree, dict_lemmas_size, grouped_heights)
%lprun -f compute_part_new_new compute_part_new_new(whole_tree, dict_lemmas_size, grouped_heights)
print('Time on calculating partial repeats: ' + str((time.time() - start) % 60))

0
0.29505109786987305
1
1.3842949867248535
2
1.6298727989196777
3
2.9113762378692627
4
3.745997190475464
5
4.040239095687866
6
0.9162311553955078
7
0.6441891193389893
8
0.6087419986724854
9
0.5605340003967285
10
0.5257279872894287
11
0.0004639625549316406
12
0.0005028247833251953
13
0.5428929328918457
14
0.5522897243499756
15
0.00029587745666503906
Time on calculating partial repeats: 18.4059419631958
